## Contexto

Em uma transportadora, temos um problema onde temos produtos com características de nome, tamanho (em metros cúbicos) e valor, assim como, temos um caminhão com um limite de tamanho. Precisamos levar o máximo de produtos que conseguirmos, porém, é necessário priorizar o valor dos produtos, ou seja, os produtos mais caros que consequentemente darão mais lucro deverão estar no caminhão sem ultrapassar o valor máximo de tamanho.

Para isso, é necessário criar um algoritmo genético onde a solução tem como objetivo **maximizar o valor de lucro** nesse transporte para a transportadora.

### Classe produtos

Primeiramente, iremos declarar a classe produto.

In [1]:
class Produto():
    def __init__(self, nome, espaco, valor):
        self.nome = nome
        self.espaco = espaco
        self.valor = valor

In [4]:
lista_produtos = []

lista_produtos.append(Produto("Geladeira Dako", 0.751, 999.90))
lista_produtos.append(Produto("Iphone 6", 0.0000899, 2911.12))
lista_produtos.append(Produto("TV 55' ", 0.400, 4346.99))
lista_produtos.append(Produto("TV 50' ", 0.290, 3999.90))
lista_produtos.append(Produto("TV 42' ", 0.200, 2999.00))
lista_produtos.append(Produto("Notebook Dell", 0.00350, 2499.90))
lista_produtos.append(Produto("Ventilador Panasonic", 0.496, 199.90))
lista_produtos.append(Produto("Microondas Electrolux", 0.0424, 308.66))
lista_produtos.append(Produto("Microondas LG", 0.0544, 429.90))
lista_produtos.append(Produto("Microondas Panasonic", 0.0319, 299.29))
lista_produtos.append(Produto("Geladeira Brastemp", 0.635, 849.00))
lista_produtos.append(Produto("Geladeira Consul", 0.870, 1199.89))
lista_produtos.append(Produto("Notebook Lenovo", 0.498, 1999.90))
lista_produtos.append(Produto("Notebook Asus", 0.527, 3999.00))

[p.nome for p in lista_produtos]

['Geladeira Dako',
 'Iphone 6',
 "TV 55' ",
 "TV 50' ",
 "TV 42' ",
 'Notebook Dell',
 'Ventilador Panasonic',
 'Microondas Electrolux',
 'Microondas LG',
 'Microondas Panasonic',
 'Geladeira Brastemp',
 'Geladeira Consul',
 'Notebook Lenovo',
 'Notebook Asus']

### Classe indivíduo

A primeira fase de um algoritmo genético é **gerar a população inicial**. A população é caracterizada por um **conjunto de indivíduos**.

geracao=0 significa que quando um indivíduo for criado, ele inicialmente não evoluiu nada.

In [ ]:
from random import random

class Individuo():
    def __init__(self, espacos, valores, limite_espacos, geracao=0):
        self.espacos = espacos # Quais são os espaços ocupados dos produtos que serão carregados (em m³)
        self.valores = valores # Valores em reais
        self.limite_espacos = limite_espacos # Limite do espaço (caminhão)
        self.nota_avaliacao = 0 # Cada indivíduo terá uma nota que definirá se ele é bom ou ruim comparado aos outros
        self.geracao = geracao
        self.cromossomo = [] # Soluções do indivíduo
        
        # Inicializando a solução aleatóriamente
        for i in range(len(espacos)):
            if random() < 0.5:
                self.cromossomo.append('0')
            else:
                self.cromossomo.append('1')